In [2]:
# All library connection
import pandas as pd
import numpy as np
import re #регулярки
from math import gcd, isclose
import sympy as sp # Библиотека символьных вычеслений, мат.задачи
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

In [4]:
# pymatgen def to obtain structures
from pymatgen.core.structure import Composition

In [5]:
from collections import defaultdict

In [22]:
#loading data
df = pd.read_csv(r'C:\Users\Ксения\Project\Magnet_Exchange_bias\part2.csv')

In [7]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 41 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Unnamed: 0                                     568 non-null    int64  
 1   id                                             568 non-null    int64  
 2   c1                                             568 non-null    object 
 3   c2                                             0 non-null      float64
 4   c3                                             0 non-null      float64
 5   formula_combined                               0 non-null      float64
 6   chemical_formula                               0 non-null      float64
 7   x                                              568 non-null    float64
 8   y                                              568 non-null    float64
 9   z                                              568 non

***Difference***
- composition - core+shell with Ni1O1 like formula
-  chemical formula - Ni50NiO50
-  Structure - alhpa, gamma

## Поработаем со столбцами

In [23]:
# Удалим все колонки, значения которых точно не подлежат востановлению
del_col = ['Exp type', 'Scale bar (nm)', 'microphotography pic','Spectrum', 'JCPDS card (core)', 'JCPDS card (shell)', 'X-ray tube (nm)', 'Instrument', 'Crystalline size', 'Material fraction', 'curie temperature (K)', 'H-range min (kOe)', 'Instrument.1', 'M-H curve', 'applied field(kOe)', 'Article id', 'Method', 'H-range min (kOe)','Tb', 'Keff (effective anisotropy constant) (erg/cm3)',
            'Keff (Jm^-3)', 'Total magnetization ', 'Atomic mass ratio', 'Number of magnetic ions per unit cell', 'Name', 'synonyms',
            'get_exchange_group_info', 'cooling fields (mT)', 'Hc of measurement (kOe) ', 'number_of_unique_magnetic_sites', 'number_of_magnetic_sites', 'Cas rn (core_1)', 'Cas rn (core_2)' ]
df = df.drop(columns=del_col)
print(df.info())

KeyError: "['Exp type', 'Scale bar (nm)', 'microphotography pic', 'Spectrum', 'JCPDS card (core)', 'JCPDS card (shell)', 'X-ray tube (nm)', 'Instrument', 'Crystalline size', 'Material fraction', 'curie temperature (K)', 'H-range min (kOe)', 'Instrument.1', 'M-H curve', 'applied field(kOe)', 'Article id', 'Method', 'H-range min (kOe)', 'Tb', 'Keff (effective anisotropy constant) (erg/cm3)', 'Keff (Jm^-3)', 'Total magnetization ', 'Atomic mass ratio', 'Number of magnetic ions per unit cell', 'Name', 'synonyms', 'get_exchange_group_info', 'cooling fields (mT)', 'Hc of measurement (kOe) ', 'number_of_unique_magnetic_sites', 'number_of_magnetic_sites', 'Cas rn (core_1)', 'Cas rn (core_2)'] not found in axis"

In [8]:
# Теперь можно удалить эти колонки, так как они имеют такие же значения как и crystal_structure(core/shell) и chemical formula
print(f'Unique cores: {df["core_1"].unique()}', f'Amount of unique cores composition: {len(df["core_1"].unique())}', sep='\n')
print(f'Unique shells: {df["core_2(shell)"].unique()}', f'Amount of unique shells composition: {len(df["core_2(shell)"].unique())}', sep='\n')

Unique cores: ['Zn1O1' 'Cu1O1' 'Mn0.50Zn0.50Fe2O4' 'La1Fe1O3' 'Ni1' 'Mg1Fe2O4'
 'Co1Fe2O4' 'Bi1Fe1O3' 'Co1' 'Fe1Rh1' 'Mn1O2' 'Ag1' 'Fe1' 'Fe1O1' 'Ni1O1'
 'Fe3O4' 'Bi2Fe4O9' 'Ti1O2' 'Co0.4Fe2.6O4' 'Co0.7Fe2.3O4' 'Fe0.7Co0.3'
 'Fe0.5Co0.5' 'Fe0.3Co0.7' 'La2Fe1Mn1O6' 'Fe1V1O4' 'Au1'
 'Bi0.9Gd0.1Fe0.9Ti0.1O3' 'Nd0.78Sr0.22CoO3' 'Fe2O3' 'La0.85Sr0.15FeO3'
 'Mn3O4' 'La0.67Sr0.33MnO3' 'Mn1O1' 'Co3O4' 'Ni0.99Fe0.01O1'
 'Ni0.98Fe0.02O1' 'Ni0.96Fe0.04O1' 'Ni0.97Fe0.03O1' 'Ag0.5Cr2.5O4'
 'Ag1Cr1O2' 'Co1O1' 'Mn1Fe2O4' 'Bi0.9La0.1FeO3' 'Nd0.90Ho0.10FeO3'
 'Dy1Cr1Ti1O5' 'Gd1Cr1Ti1O5' 'Ba1Fe12O19' 'Ni1Fe2O4' 'Co1Cr2O4' 'Fe1Ni1'
 'Mn1' 'Fe1F2' 'Fe5C2' 'Fe1Ni3' 'Fe2.2C1' 'La2Fe2O6' 'Co1P1']
Amount of unique cores composition: 57
Unique shells: ['Co1Fe2O4' '0' 'Ni1O1' 'Bi1Fe1O3' 'Co1O1' 'Fe3O4' 'Fe2O3' 'Bi2Fe4O9'
 'Сu1O1' 'Si1O2' 'Co3O4' 'Mn3O4' 'Ni1Fe2O4' 'Co1Mn2O4' 'Fe1' 'Mn1O1'
 'Fe3C1' 'Mg1O1' 'Mg1O1, Co1Fe2O4' 'Mo1' 'Co1']
Amount of unique shells composition: 21


In [9]:
df = df.drop(columns=['core_1', 'core_2(shell)']) #удалим их

In [24]:
#переименуем название колонк для удобства и преобразуем их в нижний регистр
df.columns = df.columns.str.lower()

new_column_names = {
    'id': 'id', 'crystal_structure(core/shell)': 'crystal_structure',
    'space group(core)': 'space_group_core', 'space group(shell)': 'space_group_shell',
    'chemical formula': 'chemical_formula', 'structure': 'structure',
    'length (d)': 'length_d_nm', 'width': 'width_nm',
    'depth': 'depth_nm', 'shape': 'shape', 'temperature (k)': 'temperature_k',
    'h-range max(koe)': 'h_range_max_koe', 'ms (emu/g)': 'sat_em_g',
    'hc (oe)': 'coer_oe', 'mrem (emu/g)': 'rem_emu_g', 'exchange bias shift hb or heb(oe)': 'exc_bias_oe',
    'vertical loop shift m vsl (emu/g)': 'ver_shift_emu_g', 'fc field(t)': 'fc_field_t'
}

# Переименование столбцов
df = df.rename(columns=new_column_names)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 41 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   unnamed: 0                                     568 non-null    int64  
 1   id                                             568 non-null    int64  
 2   c1                                             568 non-null    object 
 3   c2                                             0 non-null      float64
 4   c3                                             0 non-null      float64
 5   formula_combined                               0 non-null      float64
 6   chemical_formula                               0 non-null      float64
 7   x                                              568 non-null    float64
 8   y                                              568 non-null    float64
 9   z                                              568 non

## Поработаем с химическими формулами

In [68]:
chemical_formulas = []

for formula in df['chemical_formula']: # Итерация по каждой формуле в столбце 'chemical formula'
    if not formula.isalnum(): # Проверка, содержит ли формула только буквенно-цифровые символы
        line = re.split("-|,|/|@|–", formula) # Разделение формулы по символам '-', '@', '/', and ','
        components = [] # Создание пустого списка для хранения компонентов формулы
        for i in line:  # Итерация по каждому компоненту в разделенной формуле
            if i.strip(): # Добавление компонента в список, если он не пустой
                components.append(i.strip())
        chemical_formulas.append(components) # Добавление списка компонентов в список химических формул
    else:
        chemical_formulas.append([formula])  # Если формула состоит только из буквенно-цифровых символов, добавляем ее как есть

df['chemical_formula'] = chemical_formulas # Замена столбца новыми значениями

AttributeError: 'float' object has no attribute 'isalnum'

In [12]:
# Функция для разбивки списка в каждой ячейке на отдельные элементы и вставки их в новые столбцы
def split_elements(row):
    # Получаем список элементов из ячейки
    elements = row['chemical_formula']
    # Размещаем элементы в соответствующих столбцах
    for i, element in enumerate(elements):
        if i < 3:
            row[f'c{i+1}'] = element
    return row

# Применяем функцию к каждой строке датафрейма
df = df.apply(split_elements, axis=1)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   c1                 533 non-null    object 
 1   c2                 322 non-null    object 
 2   c3                 9 non-null      object 
 3   chemical_formula   533 non-null    object 
 4   coer_oe            510 non-null    float64
 5   crystal_structure  533 non-null    object 
 6   depth_nm           509 non-null    float64
 7   exc_bias_oe        500 non-null    float64
 8   fc_field_t         508 non-null    float64
 9   h_range_max_koe    526 non-null    float64
 10  id                 533 non-null    int64  
 11  length_d_nm        521 non-null    float64
 12  mr (emu/g)         430 non-null    float64
 13  sat_em_g           420 non-null    float64
 14  shape              493 non-null    object 
 15  space_group_core   532 non-null    object 
 16  space_group_shell  533 non

In [25]:
# Проверим как разделились значения списка
print(df['c1'], df['c2'], df['c3'], end='\n') # В с3 должно было быть всего 9 значений

0                AuFe2O3
1      Co0.03Fe0.97Fe2O4
2      Co0.05Fe0.95Fe2O4
3        Co0.1Fe0.9Fe2O4
4        Co0.1Fe0.9Fe2O4
             ...        
563              ZnFe2O4
564              ZnFe2O4
565              ZnFe2O4
566              ZnFe2O4
567              ZnFe2O4
Name: c1, Length: 568, dtype: object 0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
563   NaN
564   NaN
565   NaN
566   NaN
567   NaN
Name: c2, Length: 568, dtype: float64 0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
563   NaN
564   NaN
565   NaN
566   NaN
567   NaN
Name: c3, Length: 568, dtype: float64


In [12]:
print(df['c1'].info())

<class 'pandas.core.series.Series'>
RangeIndex: 568 entries, 0 to 567
Series name: c1
Non-Null Count  Dtype 
--------------  ----- 
568 non-null    object
dtypes: object(1)
memory usage: 4.6+ KB
None


In [26]:
#Сделаем костыль для нестеохео соединений 
print(df['c1'].unique(), df['c2'].unique(), df['c3'].unique(), end='\n')
non_steh = ['Co0.03Fe0.97Fe2O4', 'Co0.05Fe0.95Fe2O4', 'Co0.1Fe0.9Fe2O4',
 'Co0.24Mn0.15Fe0.61Fe2O4', 'Co0.3Fe0.7Fe2O4', 'Co0.5Fe0.5Fe2O4',
 'Co0.7Fe0.3Fe2O4', 'Fe0.5Mn0.5Fe2O4',
 'Fe2.2C1Fe5C2', 'Mg0.1Mn0.9Fe2O4',
 'Mg0.3Mn0.7Fe2O4', 'Mg0.4Mn0.6Fe2O4', 'Mg0.5Mn0.5Fe2O4',
 'Mn0.5Zn0.5Fe2O4', 'Ni0.5Zn0.5Fe2O4', 'Ni0.65Zn0.35Fe0.05Fe2O4',
 'Ni0.65Zn0.35Fe0.05Fe2O5', 'Ni0.65Zn0.35Fe0.05Fe2O6',
 'Ni0.65Zn0.35Fe0.05Fe2O7', 'NiCo2O4Au0.44',
 'Tb0.22Fe0.78Fe2O4', 'Tb0.5Fe0.5Fe2O4', 'Tb0.75Fe0.25Fe2O4',
 'Zn0.1Mn0.9Fe2O4', 'Zn0.2Mn0.8Fe2O4', 'Zn0.3Co0.7Fe2O4', 'Zn0.3Mn0.7Fe2O4',
 'Zn0.4Mn0.6Fe2O4', 'Zn0.5Mn0.5Fe2O4', 'Zn0.6Mn0.4Fe2O4', 'Zn0.7Mn0.3Fe2O4',
 'Zn0.8Mn0.2Fe2O4', 'Zn0.9Mn0.1Fe2O4']

'''
non_steh = ['Mn0.50Zn0.50Fe2O4',  'Co0.4Fe2.6O4', 'Co0.7Fe2.3O4', 'Fe0.7Co0.3', 'Fe0.5Co0.5',
 'Fe0.3Co0.7', 'Bi0.9Gd0.1Fe0.9Ti0.1O3', 'Nd0.78Sr0.22CoO3' 'Cu0.98' 'Cu0.96', 'La0.85Sr0.15FeO3', 
 'La0.67Sr0.33MnO3', 'Ni0.99Fe0.01O', 'Ni0.98Fe0.02O', 'Ni0.96Fe0.04O', 'Ni0.97Fe0.03O', 'Ag0.5Cr2.5O4',
 'Bi0.9La0.1FeO3', 'Fe2.2C', 'Fe0.02O', 'Fe0.04O']
'''

['AuFe2O3' 'Co0.03Fe0.97Fe2O4' 'Co0.05Fe0.95Fe2O4' 'Co0.1Fe0.9Fe2O4'
 'Co0.24Mn0.15Fe0.61Fe2O4' 'Co0.3Fe0.7Fe2O4' 'Co0.5Fe0.5Fe2O4'
 'Co0.7Fe0.3Fe2O4' 'CoFe2O4' 'CuFe2O4' 'Cu2Fe2O4' 'Fe0.5Mn0.5Fe2O4'
 'Fe3O4' 'Fe2.2C1Fe5C2' 'Fe2O3' 'Fe2O3Fe4(Fe(CN)6)3' 'Fe2O3Fe4(Fe(CN)6)4'
 'Fe3O4CeO2' 'Fe3O4MnO2' 'Fe3O4Si' 'FeV2O4' 'Mg0.1Mn0.9Fe2O4'
 'Mg0.3Mn0.7Fe2O4' 'Mg0.4Mn0.6Fe2O4' 'Mg0.5Mn0.5Fe2O4' 'MgFe2O4'
 'Mn0.5Zn0.5Fe2O4' 'MnFe2O4' 'Ni0.5Zn0.5Fe2O4' 'Ni0.65Zn0.35Fe0.05Fe2O4'
 'Ni0.65Zn0.35Fe0.05Fe2O5' 'Ni0.65Zn0.35Fe0.05Fe2O6'
 'Ni0.65Zn0.35Fe0.05Fe2O7' 'NiFe2O4' 'NiCo2O4Au0.44' 'NiTe' 'PdFe2O3'
 'Tb0.22Fe0.78Fe2O4' 'Tb0.5Fe0.5Fe2O4' 'Tb0.75Fe0.25Fe2O4'
 'Zn0.1Mn0.9Fe2O4' 'Zn0.2Mn0.8Fe2O4' 'Zn0.3Co0.7Fe2O4' 'Zn0.3Mn0.7Fe2O4'
 'Zn0.4Mn0.6Fe2O4' 'Zn0.5Mn0.5Fe2O4' 'Zn0.6Mn0.4Fe2O4' 'Zn0.7Mn0.3Fe2O4'
 'Zn0.8Mn0.2Fe2O4' 'Zn0.9Mn0.1Fe2O4' 'ZnFe2O4'] [nan] [nan]


"\nnon_steh = ['Mn0.50Zn0.50Fe2O4',  'Co0.4Fe2.6O4', 'Co0.7Fe2.3O4', 'Fe0.7Co0.3', 'Fe0.5Co0.5',\n 'Fe0.3Co0.7', 'Bi0.9Gd0.1Fe0.9Ti0.1O3', 'Nd0.78Sr0.22CoO3' 'Cu0.98' 'Cu0.96', 'La0.85Sr0.15FeO3', \n 'La0.67Sr0.33MnO3', 'Ni0.99Fe0.01O', 'Ni0.98Fe0.02O', 'Ni0.96Fe0.04O', 'Ni0.97Fe0.03O', 'Ag0.5Cr2.5O4',\n 'Bi0.9La0.1FeO3', 'Fe2.2C', 'Fe0.02O', 'Fe0.04O']\n"

In [28]:
# Создаем список для сохранения результатов
normalized_formulas = []

# Применяем функцию к каждому элементу списка non_steh
for formula in non_steh:
    comp_before = Composition(formula)  # Создаем объект Composition до применения функции
    norm_formula, factor = comp_before.get_integer_formula_and_factor()
    comp_after = norm_formula  # Создаем объект Composition после применения функции
    normalized_formulas.append((comp_before, comp_after, factor))

# Выводим результаты до/после
for comp_before, comp_after, f in normalized_formulas:
    print(f"Original: {comp_before}, Normalized: {comp_after}, Factor: {f}")

Original: Co0.03 Fe2.97 O4, Normalized: Fe297Co3O400, Factor: 0.00999999999996265
Original: Co0.05 Fe2.95 O4, Normalized: Fe59CoO80, Factor: 0.04999999999999978
Original: Co0.1 Fe2.9 O4, Normalized: Fe29CoO40, Factor: 0.09999999999999976
Original: Co0.24 Mn0.15 Fe2.61 O4, Normalized: Mn15Fe261(Co3O50)8, Factor: 0.009999999999992293
Original: Co0.3 Fe2.7 O4, Normalized: Fe27Co3O40, Factor: 0.0999999999999997
Original: Co0.5 Fe2.5 O4, Normalized: Fe5CoO8, Factor: 0.5
Original: Co0.7 Fe2.3 O4, Normalized: Fe23Co7O40, Factor: 0.09999999999999987
Original: Fe2.5 Mn0.5 O4, Normalized: MnFe5O8, Factor: 0.5
Original: Fe7.2 C3, Normalized: Fe12C5, Factor: 0.5999999999999996
Original: Mg0.1 Mn0.9 Fe2 O4, Normalized: MgMn9(FeO2)20, Factor: 0.09999999999999998
Original: Mg0.3 Mn0.7 Fe2 O4, Normalized: Mg3Mn7(FeO2)20, Factor: 0.09999999999999998
Original: Mg0.4 Mn0.6 Fe2 O4, Normalized: Mg2Mn3(FeO2)10, Factor: 0.19999999999999996
Original: Mg0.5 Mn0.5 Fe2 O4, Normalized: MgMn(FeO2)4, Factor: 0.5
Or

In [29]:
# Костыль для таких соединений
non_stoich_compounds_dict = {
    'Co0.03Fe0.97Fe2O4': 'Fe297Co3O400', 'Co0.05Fe0.95Fe2O4': 'Fe59CoO80', 'Co0.1Fe0.9Fe2O4': 'Fe29CoO40',
    'Co0.24Mn0.15Fe0.61Fe2O4': 'Mn15Fe261(Co3O50)8', 'Co0.3Fe0.7Fe2O4': 'Fe27Co3O40', 'Co0.5Fe0.5Fe2O4': 'Fe5CoO8',
    'Co0.7Fe0.3Fe2O4': 'Fe23Co7O40', 'Fe0.5Mn0.5Fe2O4': 'MnFe5O8',
    'Fe2.2C1Fe5C2': 'Fe12C5', 'Mg0.1Mn0.9Fe2O4': 'MgMn9(FeO2)20',
    'Mg0.3Mn0.7Fe2O4': 'Mg3Mn7(FeO2)20', 'Mg0.4Mn0.6Fe2O4': 'Mg2Mn3(FeO2)10', 'Mg0.5Mn0.5Fe2O4': 'MgMn(FeO2)4',
    'Mn0.5Zn0.5Fe2O4': 'MnZn(FeO2)4', 'Ni0.5Zn0.5Fe2O4': 'ZnFe4NiO8', 'Ni0.65Zn0.35Fe0.05Fe2O4': 'Zn7Fe41Ni13O80',
    'Ni0.65Zn0.35Fe0.05Fe2O5': 'Zn7Fe41Ni13O100', 'Ni0.65Zn0.35Fe0.05Fe2O6': 'Zn7Fe41Ni13O120',
    'Ni0.65Zn0.35Fe0.05Fe2O7': 'Zn7Fe41Ni13O140', 'NiCo2O4Au0.44': 'Co50Ni25Au11O100',
    'Tb0.22Fe0.78Fe2O4': 'Tb11Fe139O200', 'Tb0.5Fe0.5Fe2O4': 'TbFe5O8', 'Tb0.75Fe0.25Fe2O4': 'Tb3Fe9O16',
    'Zn0.1Mn0.9Fe2O4': 'Mn9Zn(FeO2)20', 'Zn0.2Mn0.8Fe2O4': 'Mn4Zn(FeO2)10', 'Zn0.3Co0.7Fe2O4': 'Zn3Fe20Co7O40', 'Zn0.3Mn0.7Fe2O4': 'Mn7Zn3(FeO2)20',
    'Zn0.4Mn0.6Fe2O4': 'Mn3Zn2(FeO2)10', 'Zn0.5Mn0.5Fe2O4': 'MnZn(FeO2)4', 'Zn0.6Mn0.4Fe2O4': 'Mn2Zn3(FeO2)10', 'Zn0.7Mn0.3Fe2O4': 'Mn3Zn7(FeO2)20',
    'Zn0.8Mn0.2Fe2O4': 'MnZn4(FeO2)10', 'Zn0.9Mn0.1Fe2O4': 'MnZn9(FeO2)20'
}

In [34]:
def replace_non_stoichiometric(column, non_stoich_compounds_dict):
    '''Функция заменяет значения ключей на значения для этих ключей'''
    for i, value in enumerate(column):
        if value != 0 and value in non_stoich_compounds_dict:
            column.at[i] = non_stoich_compounds_dict[value]
    return column

In [35]:
# Применение функции к каждому столбцу
new_df_c1 = replace_non_stoichiometric(df['c1'], non_stoich_compounds_dict)
new_df_c2 = replace_non_stoichiometric(df['c2'], non_stoich_compounds_dict)
new_df_c3 = replace_non_stoichiometric(df['c3'], non_stoich_compounds_dict)

# Вывод уникальных значений после применения функции
unique_values_c1 = new_df_c1.unique()
unique_values_c2 = new_df_c2.unique()
unique_values_c3 = new_df_c3.unique()

In [36]:
print(f"Уникальные значения в столбце c1 после применения функции:{unique_values_c1}\n Уникальные значения в столбце c2 после применения функции:{unique_values_c2}\nУникальныe значения в столбце c3 после применения функции:{unique_values_c3}", end='\n')

Уникальные значения в столбце c1 после применения функции:['AuFe2O3' 'Fe297Co3O400' 'Fe59CoO80' 'Fe29CoO40' 'Mn15Fe261(Co3O50)8'
 'Fe27Co3O40' 'Fe5CoO8' 'Fe23Co7O40' 'CoFe2O4' 'CuFe2O4' 'Cu2Fe2O4'
 'MnFe5O8' 'Fe3O4' 'Fe12C5' 'Fe2O3' 'Fe2O3Fe4(Fe(CN)6)3'
 'Fe2O3Fe4(Fe(CN)6)4' 'Fe3O4CeO2' 'Fe3O4MnO2' 'Fe3O4Si' 'FeV2O4'
 'MgMn9(FeO2)20' 'Mg3Mn7(FeO2)20' 'Mg2Mn3(FeO2)10' 'MgMn(FeO2)4' 'MgFe2O4'
 'MnZn(FeO2)4' 'MnFe2O4' 'ZnFe4NiO8' 'Zn7Fe41Ni13O80' 'Zn7Fe41Ni13O100'
 'Zn7Fe41Ni13O120' 'Zn7Fe41Ni13O140' 'NiFe2O4' 'Co50Ni25Au11O100' 'NiTe'
 'PdFe2O3' 'Tb11Fe139O200' 'TbFe5O8' 'Tb3Fe9O16' 'Mn9Zn(FeO2)20'
 'Mn4Zn(FeO2)10' 'Zn3Fe20Co7O40' 'Mn7Zn3(FeO2)20' 'Mn3Zn2(FeO2)10'
 'Mn2Zn3(FeO2)10' 'Mn3Zn7(FeO2)20' 'MnZn4(FeO2)10' 'MnZn9(FeO2)20'
 'ZnFe2O4']
 Уникальные значения в столбце c2 после применения функции:[nan]
Уникальныe значения в столбце c3 после применения функции:[nan]


In [99]:
from pymatgen.core.composition import Composition
import pandas as pd

def get_normalized_formulas(formulas):
    normalized_formulas = {}
    for formula in formulas:
        comp_before = Composition(formula)
        norm_formula, factor = comp_before.get_integer_formula_and_factor()
        comp_after = Composition(norm_formula)
        normalized_formulas[comp_before] = comp_after
    return normalized_formulas

# Пример использования
unique_values_c1 = ['AuFe2O3', 'Co0.03Fe0.97Fe2O4', 'Co0.05Fe0.95Fe2O4', 'Co0.1Fe0.9Fe2O4',
 'Co0.24Mn0.15Fe0.61Fe2O4', 'Co0.3Fe0.7Fe2O4', 'Co0.5Fe0.5Fe2O4',
 'Co0.7Fe0.3Fe2O4', 'CoFe2O4', 'CuFe2O4', 'Cu2Fe2O4', 'Fe0.5Mn0.5Fe2O4',
 'Fe3O4', 'Fe2.2C1Fe5C2', 'Fe2O3', 'Fe2O3Fe4(Fe(CN)6)3', 'Fe2O3Fe4(Fe(CN)6)4',
 'Fe3O4CeO2', 'Fe3O4MnO2', 'Fe3O4Si', 'FeV2O4', 'Mg0.1Mn0.9Fe2O4',
 'Mg0.3Mn0.7Fe2O4', 'Mg0.4Mn0.6Fe2O4', 'Mg0.5Mn0.5Fe2O4', 'MgFe2O4',
 'Mn0.5Zn0.5Fe2O4', 'MnFe2O4', 'Ni0.5Zn0.5Fe2O4', 'Ni0.65Zn0.35Fe0.05Fe2O4',
 'Ni0.65Zn0.35Fe0.05Fe2O5', 'Ni0.65Zn0.35Fe0.05Fe2O6',
 'Ni0.65Zn0.35Fe0.05Fe2O7', 'NiFe2O4', 'NiCo2O4Au0.44', 'NiTe', 'PdFe2O3',
 'Tb0.22Fe0.78Fe2O4', 'Tb0.5Fe0.5Fe2O4', 'Tb0.75Fe0.25Fe2O4',
 'Zn0.1Mn0.9Fe2O4', 'Zn0.2Mn0.8Fe2O4', 'Zn0.3Co0.7Fe2O4', 'Zn0.3Mn0.7Fe2O4',
 'Zn0.4Mn0.6Fe2O4', 'Zn0.5Mn0.5Fe2O4', 'Zn0.6Mn0.4Fe2O4', 'Zn0.7Mn0.3Fe2O4',
 'Zn0.8Mn0.2Fe2O4', 'Zn0.9Mn0.1Fe2O4', 'ZnFe2O4']

# Замена значений
normalized_values = get_normalized_formulas(unique_values_c1)

# Вывод результатов
for comp_before, comp_after in normalized_values.items():
    print(f"Original: {comp_before}, Normalized: {comp_after}")

Original: Au1 Fe2 O3, Normalized: Fe2 Au1 O3
Original: Co0.03 Fe2.97 O4, Normalized: Fe297 Co3 O400
Original: Co0.05 Fe2.95 O4, Normalized: Fe59 Co1 O80
Original: Co0.1 Fe2.9 O4, Normalized: Fe29 Co1 O40
Original: Co0.24 Mn0.15 Fe2.61 O4, Normalized: Mn15 Fe261 Co24 O400
Original: Co0.3 Fe2.7 O4, Normalized: Fe27 Co3 O40
Original: Co0.5 Fe2.5 O4, Normalized: Fe5 Co1 O8
Original: Co0.7 Fe2.3 O4, Normalized: Fe23 Co7 O40
Original: Co1 Fe2 O4, Normalized: Fe2 Co1 O4
Original: Cu1 Fe2 O4, Normalized: Fe2 Cu1 O4
Original: Cu2 Fe2 O4, Normalized: Fe1 Cu1 O2
Original: Fe2.5 Mn0.5 O4, Normalized: Mn1 Fe5 O8
Original: Fe3 O4, Normalized: Fe3 O4
Original: Fe7.2 C3, Normalized: Fe12 C5
Original: Fe2 O3, Normalized: Fe2 O3
Original: Fe9 O3 C18 N18, Normalized: Fe3 C6 N6 O1
Original: Fe10 O3 C24 N24, Normalized: Fe10 C24 N24 O3
Original: Fe3 O6 Ce1, Normalized: Ce1 Fe3 O6
Original: Fe3 O6 Mn1, Normalized: Mn1 Fe3 O6
Original: Fe3 O4 Si1, Normalized: Fe3 Si1 O4
Original: Fe1 V2 O4, Normalized: V2 Fe

In [30]:
copy_formula = df['c1']
df['orig_formula'] = copy_formula 

In [31]:
df['orig_formula'] = copy_formula 

In [37]:
df['c1']

0           AuFe2O3
1      Fe297Co3O400
2         Fe59CoO80
3         Fe29CoO40
4         Fe29CoO40
           ...     
563         ZnFe2O4
564         ZnFe2O4
565         ZnFe2O4
566         ZnFe2O4
567         ZnFe2O4
Name: c1, Length: 568, dtype: object

### Время для использования Composition

Однако в наших колонках есть пропуски и 0, поэттому надо создать функцию, которая будет игнорировать такие строки

In [78]:
def process_column(df, column_name):
    """
    Функция проверяет каждую ячейку в столбце DataFrame на равенство 0 или NaN.
    Если значение не равно 0 или NaN, оно заменяется на результат применения класса Composition из библиотеки pymatgen.

    Args:
        df (pandas.DataFrame): Исходный DataFrame.
        column_name (str): Имя столбца, который нужно обработать.

    Returns:
        pandas.DataFrame: DataFrame c обработанным столбцом.
    """
    new_column = []
    for value in df[column_name]:
        if pd.notna(value) and value != 0:
            # Если значение не NaN и не равно 0, применяем класс Composition
            comp = Composition(str(value))
            new_value = comp.formula
        else:
            new_value = value
        new_column.append(new_value)

    df[column_name] = new_column
    return df

Теперь у меня есть работающая функция composition для столбцов с nan и 0

In [79]:
# Применим на наши колонки c1, c2, c3
processed_df = process_column(df, column_name='c1')
processed_df = process_column(df, column_name='c2')
processed_df = process_column(df, column_name='c3')
print(df['c3'].unique())
print(df['c2'].unique())

[nan]
[nan]


In [80]:
print(df['c1'].unique())

['Fe2 Au1 O3' 'Fe2.97 Co0.03 O4' 'Fe2.95 Co0.05 O4' 'Fe2.9 Co0.1 O4'
 'Mn0.15 Fe2.61 Co0.24 O4' 'Fe2.7 Co0.3 O4' 'Fe2.5 Co0.5 O4'
 'Fe2.3 Co0.7 O4' 'Fe2 Co1 O4' 'Fe2 Cu1 O4' 'Fe2 Cu2 O4' 'Mn0.5 Fe2.5 O4'
 'Fe3 O4' 'Fe7.2 C3' 'Fe2 O3' 'Fe9 C18 N18 O3' 'Fe10 C24 N24 O3'
 'Ce1 Fe3 O6' 'Mn1 Fe3 O6' 'Fe3 Si1 O4' 'V2 Fe1 O4' 'Mg0.1 Mn0.9 Fe2 O4'
 'Mg0.3 Mn0.7 Fe2 O4' 'Mg0.4 Mn0.6 Fe2 O4' 'Mg0.5 Mn0.5 Fe2 O4'
 'Mg1 Fe2 O4' 'Mn0.5 Zn0.5 Fe2 O4' 'Mn1 Fe2 O4' 'Zn0.5 Fe2 Ni0.5 O4'
 'Zn0.35 Fe2.05 Ni0.65 O4' 'Zn0.35 Fe2.05 Ni0.65 O5'
 'Zn0.35 Fe2.05 Ni0.65 O6' 'Zn0.35 Fe2.05 Ni0.65 O7' 'Fe2 Ni1 O4'
 'Co2 Ni1 Au0.44 O4' 'Ni1 Te1' 'Fe2 Pd1 O3' 'Tb0.22 Fe2.78 O4'
 'Tb0.5 Fe2.5 O4' 'Tb0.75 Fe2.25 O4' 'Mn0.9 Zn0.1 Fe2 O4'
 'Mn0.8 Zn0.2 Fe2 O4' 'Zn0.3 Fe2 Co0.7 O4' 'Mn0.7 Zn0.3 Fe2 O4'
 'Mn0.6 Zn0.4 Fe2 O4' 'Mn0.4 Zn0.6 Fe2 O4' 'Mn0.3 Zn0.7 Fe2 O4'
 'Mn0.2 Zn0.8 Fe2 O4' 'Mn0.1 Zn0.9 Fe2 O4' 'Zn1 Fe2 O4']


Создадим отдельный столбец, где будем содержать полностью материал созданный из трёх колонок c1, c2, c3

In [81]:
# Функция для сложения формул из нескольких столбцов
def combine_compositions(row):
    compositions = []
    for column in ['c1', 'c2', 'c3']:
        value = row[column]
        if pd.notnull(value):  # Проверяем, не пустое ли значение
            compositions.append(value)
    return ' '.join(compositions)

# Создаем новый столбец с объединенными формулами
df['formula_combined'] = df.apply(combine_compositions, axis=1)

In [82]:
print(df['formula_combined']) # Получилось немного криво, но не критично я думаю можно исправить, у меня встрчеаются повторяющиеся соедиения и индексы их не скаладываются

0            Fe2 Au1 O3
1      Fe2.97 Co0.03 O4
2      Fe2.95 Co0.05 O4
3        Fe2.9 Co0.1 O4
4        Fe2.9 Co0.1 O4
             ...       
563          Zn1 Fe2 O4
564          Zn1 Fe2 O4
565          Zn1 Fe2 O4
566          Zn1 Fe2 O4
567          Zn1 Fe2 O4
Name: formula_combined, Length: 568, dtype: object


## Теперь поработаем с space_group_core и space_group_shell

In [26]:
# Преобразование значений в нижний регистр на всякий случай
df['space_group_core'] = df['space_group_core'].str.lower()
df['space_group_shell'] = df['space_group_shell'].str.lower()

In [27]:
#посмотрим какие структуры у нас есть
print(f"Уникальные значение ядер: {df['space_group_core'].unique()}")
print(f"Уникальные значения оболочек: {df['space_group_shell'].unique()}")

Уникальные значение ядер: ['p63mc' 'fm-3m' 'fd-3m' 'pnma' 'r3c' 'p63/mmc' 'pm-3m' 'i4/m' 'im-3m'
 'pbam' 'p42/mnm' 'p1' nan 'i41/amd' 'p-3m1' 'r3m' 'p4332' 'p4/mmm'
 'i-43m' 'cmcm' 'p4132' 'c2/c' 'p6322' 'pnnm']
Уникальные значения оболочек: ['fd-3m' '0' 'fm-3m' 'r3c' 'p4332' 'pbam' 'p4132' 'pnma' 'i41/amd'
 'p41212' 'im-3m' 'p63/mmc' 'p42/mnm']


[ 'fm-3m' 'fd-3m' 'pbnm' - заменить на pnma 'r3c''pm-3m' 'im-3m' 'i-43m'
  'p-3m1'

In [28]:
# Надо заменить на значения с чертой/ - если значение равно 221-230 или 215-220 или 200-206 или 187-190 или 162-167

Осторожней с 62 - так она может пиcаться ещё как 'Pnma' и 'pbnm'

In [29]:
# Создадим словарь, чтобы перевести простаранственные группы в числовые обозначения пространнственных групп
dict_space_group = {
    1: "P1", 2: "P-1", 3: "P121", 4: "P1211", 5: "C121", 6: "P1m1", 7: "P1c1", 8: "C1m1",
    9: "C1c1", 10: "P12/m1", 11: "P121/m1", 12: "C12/m1", 13: "P12/c1", 14: "P121/c1",
    15: "C2/c", 16: "P222", 17: "P2221", 18: "P21212", 19: "P212121", 20: "C2221",
    21: "C222", 22: "F222", 23: "I222", 24: "I212121", 25: "Pmm2", 26: "Pmc21", 27: "Pcc2",
    28: "Pma2", 29: "Pca21", 30: "Pnc2", 31: "Pmn21", 32: "Pba2", 33: "Pna21", 34: "Pnn2",
    35: "Cmm2", 36: "Cmc21", 37: "Ccc2", 38: "Amm2", 39: "Abm2", 40: "Ama2", 41: "Aba2",
    42: "Fmm2", 43: "Fdd2", 44: "Imm2", 45: "Iba2", 46: "Ima2", 47: "Pmmm", 48: "Pnnn",
    49: "Pccm", 50: "Pban", 51: "Pmma", 52: "Pnna", 53: "Pmna", 54: "Pcca", 55: "Pbam",
    56: "Pccn", 57: "Pbcm", 58: "Pnnm", 59: "Pmmn", 60: "Pbcn", 61: "Pbca", 62: "Pnma",
    63: "Cmcm", 64: "Cmca", 65: "Cmmm", 66: "Cccm", 67: "Cmma", 68: "Ccca", 69: "Fmmm",
    70: "Fddd", 71: "Immm", 72: "Ibam", 73: "Ibca", 74: "Imma", 75: "P4", 76: "P41",
    77: "P42", 78: "P43", 79: "I4", 80: "I41", 81: "P-4", 82: "I-4", 83: "P4/m", 84: "P42/m",
    85: "P4/n", 86: "P42/n", 87: "I4/m", 88: "I41/a", 89: "P422", 90: "P4212", 91: "P4122",
    92: "P41212", 93: "P4222", 94: "P42212", 95: "P4322", 96: "P43212", 97: "I422",
    98: "I4122", 99: "P4mm", 100: "P4bm", 101: "P42cm", 102: "P42nm", 103: "P4cc",
    104: "P4nc", 105: "P42mc", 106: "P42bc", 107: "I4mm", 108: "I4cm", 109: "I41md",
    110: "I41cd", 111: "P-42m", 112: "P-42c", 113: "P-421m", 114: "P-421c", 115: "P-4m2",
    116: "P-4c2", 117: "P-4b2", 118: "P-4n2", 119: "I-4m2", 120: "I-4c2", 121: "I-42m",
    122: "I-42d", 123: "P4/mmm", 124: "P4/mcc", 125: "P4/nbm", 126: "P4/nnc", 127: "P4/mbm",
    128: "P4/mnc", 129: "P4/nmm", 130: "P4/ncc", 131: "P42/mmc", 132: "P42/mcm",
    133: "P42/nbc", 134: "P42/nnm", 135: "P42/mbc", 136: "P42/mnm", 137: "P42/nmc",
    138: "P42/ncm", 139: "I4/mmm", 140: "I4/mcm", 141: "I41/amd", 142: "I41/acd",
    143: "P3", 144: "P31", 145: "P32", 146: "R3", 147: "P-3", 148: "R-3", 149: "P312",
    150: "P321", 151: "P3112", 152: "P3121", 153: "P3212", 154: "P3221", 155: "R32",
    156: "P3m1", 157: "P31m", 158: "P3c1", 159: "P31c", 160: "R3m", 161: "R3c", 162: "P-31m",
    163: "P-31c", 164: "P-3m1", 165: "P-3c1", 166: "R-3m", 167: "R-3c", 168: "P6", 169: "P61",
    170: "P65", 171: "P62", 172: "P64", 173: "P63", 174: "P-6", 175: "P6/m", 176: "P63/m",
    177: "P622", 178: "P6122", 179: "P6522", 180: "P6222", 181: "P6422", 182: "P6322",
    183: "P6mm", 184: "P6cc", 185: "P63cm", 186: "P63mc", 187: "P-6m2", 188: "P-6c2",
    189: "P-62m", 190: "P-62c", 191: "P6/mmm", 192: "P6/mcc", 193: "P63/mcm", 194: "P63/mmc",
    195: "P23", 196: "F23", 197: "I23", 198: "P213", 199: "I213", 200: "Pm-3", 201: "Pn-3",
    202: "Fm-3", 203: "Fd-3", 204: "Im-3", 205: "Pa-3", 206: "Ia-3", 207: "P432", 208: "P4232",
    209: "F432", 210: "F4132", 211: "I432", 212: "P4332", 213: "P4132", 214: "I4132",
    215: "P-43m", 216: "F-43m", 217: "I-43m", 218: "P-43n", 219: "F-43c", 220: "I-43d",
    221: "Pm-3m", 222: "Pn-3n", 223: "Pm-3n", 224: "Pn-3m", 225: "Fm-3m", 226: "Fm-3c",
    227: "Fd-3m", 228: "Fd-3c", 229: "Im-3m", 230: "Ia-3d"
}

In [30]:
# Создание словаря со значениями в нижнем регистре для сравнени по нему
dict_space_group_lower = {key: value.lower() for key, value in dict_space_group.items()}

In [31]:
# Чтобы проще было проверять и менять значения
dict_space_group_swapped = {v: k for k, v in dict_space_group_lower.items()}

In [32]:
print(dict_space_group_swapped)

{'p1': 1, 'p-1': 2, 'p121': 3, 'p1211': 4, 'c121': 5, 'p1m1': 6, 'p1c1': 7, 'c1m1': 8, 'c1c1': 9, 'p12/m1': 10, 'p121/m1': 11, 'c12/m1': 12, 'p12/c1': 13, 'p121/c1': 14, 'c2/c': 15, 'p222': 16, 'p2221': 17, 'p21212': 18, 'p212121': 19, 'c2221': 20, 'c222': 21, 'f222': 22, 'i222': 23, 'i212121': 24, 'pmm2': 25, 'pmc21': 26, 'pcc2': 27, 'pma2': 28, 'pca21': 29, 'pnc2': 30, 'pmn21': 31, 'pba2': 32, 'pna21': 33, 'pnn2': 34, 'cmm2': 35, 'cmc21': 36, 'ccc2': 37, 'amm2': 38, 'abm2': 39, 'ama2': 40, 'aba2': 41, 'fmm2': 42, 'fdd2': 43, 'imm2': 44, 'iba2': 45, 'ima2': 46, 'pmmm': 47, 'pnnn': 48, 'pccm': 49, 'pban': 50, 'pmma': 51, 'pnna': 52, 'pmna': 53, 'pcca': 54, 'pbam': 55, 'pccn': 56, 'pbcm': 57, 'pnnm': 58, 'pmmn': 59, 'pbcn': 60, 'pbca': 61, 'pnma': 62, 'cmcm': 63, 'cmca': 64, 'cmmm': 65, 'cccm': 66, 'cmma': 67, 'ccca': 68, 'fmmm': 69, 'fddd': 70, 'immm': 71, 'ibam': 72, 'ibca': 73, 'imma': 74, 'p4': 75, 'p41': 76, 'p42': 77, 'p43': 78, 'i4': 79, 'i41': 80, 'p-4': 81, 'i-4': 82, 'p4/m': 8

In [33]:
def replace_space_group_values(df, dict_space_group_swapped):
    for column in ['space_group_core', 'space_group_shell']:
        for i, value in enumerate(df[column]):
            if pd.notnull(value) and value != 0:
                # Проверяем наличие значения в словаре и заменяем его, если оно есть
                if value in dict_space_group_swapped:
                    df.at[i, column] = dict_space_group_swapped[value]
    return df

# Применяем функцию к датафрейму
new_df = replace_space_group_values(df, dict_space_group_swapped)

In [34]:
# Теперь все наши значения закодированы - не все есть моменты, которые стоит исправить - может там русские буквы
print(f"Для ядра: {df['space_group_core'].unique()}", f"Для оболочки: {df['space_group_shell'].unique()}", sep='\n')

Для ядра: [186 225 227 62 161 194 221 87 229 55 136 1 nan 141 164 160 212 123 217 63
 213 15 182 58]
Для оболочки: [227 '0' 225 161 212 55 213 62 141 92 229 194 136]


In [35]:
# Для проверки первого столбца
dict_crystal_structure = {
    "triclinic":[1, 2], 
    "monoclinic":[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    "orthorhombic":[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74], 
    "tetragonal":[75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142], 
    "trigonal":[143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167], 
    "hexagonal":[168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194],
    "cubic":[195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230]
}               

# Геометрические признаки 

Посмотрим на парметры формы и на x/y/z(nm) параметры частиц.
1. Закодируем названия форм, но сначала удалим одни и те же формы, но написанные по разному
2. Найдём площадь и объем наших частиц
3. Посмотрим на сферичность
4. Посмотрим на разброс классов

In [36]:
print(df.info()) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   c1                 533 non-null    object 
 1   c2                 322 non-null    object 
 2   c3                 9 non-null      object 
 3   chemical_formula   533 non-null    object 
 4   coer_oe            510 non-null    float64
 5   crystal_structure  533 non-null    object 
 6   depth_nm           509 non-null    float64
 7   exc_bias_oe        500 non-null    float64
 8   fc_field_t         508 non-null    float64
 9   h_range_max_koe    526 non-null    float64
 10  id                 533 non-null    int64  
 11  length_d_nm        521 non-null    float64
 12  mr (emu/g)         430 non-null    float64
 13  sat_em_g           420 non-null    float64
 14  shape              493 non-null    object 
 15  space_group_core   532 non-null    object 
 16  space_group_shell  533 non

In [37]:
print(df['shape'].unique()) # Как можно заметить многие формы являются одним и тем же, сгруппируем

['sphere' 'polyhedron' 'tetrahedral' 'octahedral' nan 'wire' 'flower'
 'egg' 'rod' 'quasi-spherical' 'cube' 'octopod' 'octahedral ' 'cubic'
 'triangle' 'hexahedron' 'quasi-sphere' 'polyhedral' 'pseudo-spherical'
 'pseudospherically' 'hexagonal' 'octahedron']


In [38]:
# Составим словарь для форм
dict_shape = { 1: ['sphere', 'quasi-spherical', 'quasi-sphere', 'pseudo-spherical', 'pseudospherically', 'egg', 'flower', 'polyhedron', 'polyhedral'], 
               2: ['octahedral', 'octahedron', 'octahedral '], 
               3: ['cubic', 'cube', 'hexahedron'], 
               4: ['hexagonal'], # hexagonal написано в статье, поэтому считаем как плоскую фигуру
               5: ['tetrahedral', 'triangle', 'tetrahedron'], 
               6: ['wire', 'rod', 'whisker'], 
               7: ['octopod', 'octopode']
}

In [39]:
# Перевеём значения формы в цифры
for key, values in dict_shape.items():
    df['shape'] = df['shape'].replace(values, key)

In [40]:
print(df['shape'].unique())

[ 1.  5.  2. nan  6.  3.  7.  4.]


# Поработаем с пропусками

In [83]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 41 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   unnamed: 0                                     568 non-null    int64  
 1   id                                             568 non-null    int64  
 2   c1                                             568 non-null    object 
 3   c2                                             0 non-null      float64
 4   c3                                             0 non-null      float64
 5   formula_combined                               568 non-null    object 
 6   chemical_formula                               0 non-null      float64
 7   x                                              568 non-null    float64
 8   y                                              568 non-null    float64
 9   z                                              568 non

In [42]:
# Создадим копию датасета
df_copy = df.copy()
df_copy  = df_copy.drop(columns=['c1', 'c2', 'c3', 'chemical_formula', 'crystal_structure', 'depth_nm', 'space_group_shell', 'structure', 'structure_', 'formula_combined'])
print(df_copy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   coer_oe           510 non-null    float64
 1   exc_bias_oe       500 non-null    float64
 2   fc_field_t        508 non-null    float64
 3   h_range_max_koe   526 non-null    float64
 4   id                533 non-null    int64  
 5   length_d_nm       521 non-null    float64
 6   mr (emu/g)        430 non-null    float64
 7   sat_em_g          420 non-null    float64
 8   shape             493 non-null    float64
 9   space_group_core  532 non-null    object 
 10  temperature_k     528 non-null    float64
 11  ver_shift_emu_g   441 non-null    float64
 12  width_nm          521 non-null    float64
dtypes: float64(11), int64(1), object(1)
memory usage: 54.3+ KB
None


In [43]:
# Преобразование типа данных в числовой
df_copy['shape'] = df_copy['shape'].astype('float64')
df_copy['space_group_core'] = df_copy['space_group_core'].astype('float64')

# Заполнение пропущенных значений в категориальных признаках модой
df_copy['shape'] = df_copy['shape'].fillna(df_copy['shape'].mode()[0])
df_copy['space_group_core'] = df_copy['space_group_core'].fillna(df_copy['space_group_core'].mode()[0])

# Преобразование типа данных в категориальный
df_copy['shape'] = df_copy['shape'].astype('category')
df_copy['space_group_core'] = df_copy['space_group_core'].astype('category')

In [44]:
df_copy['shape'].unique()

C:\Users\Ксения\AppData\Roaming\Python\Python311\site-packages\IPython\lib\pretty.py:779: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  output = repr(obj)


[1.0, 5.0, 2.0, 6.0, 3.0, 7.0, 4.0]
Categories (7, float64): [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]

In [45]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   coer_oe           510 non-null    float64 
 1   exc_bias_oe       500 non-null    float64 
 2   fc_field_t        508 non-null    float64 
 3   h_range_max_koe   526 non-null    float64 
 4   id                533 non-null    int64   
 5   length_d_nm       521 non-null    float64 
 6   mr (emu/g)        430 non-null    float64 
 7   sat_em_g          420 non-null    float64 
 8   shape             533 non-null    category
 9   space_group_core  533 non-null    category
 10  temperature_k     528 non-null    float64 
 11  ver_shift_emu_g   441 non-null    float64 
 12  width_nm          521 non-null    float64 
dtypes: category(2), float64(10), int64(1)
memory usage: 48.0 KB


In [46]:
# Преобразуем тип столбцов из object в float64
df_copy['coer_oe'] = df_copy['coer_oe'].astype('float64')
df_copy['sat_em_g'] = df_copy['sat_em_g'].astype('float64')

df_copy['sat_em_g'].unique()

array([        nan, 2.29000e+00, 2.43000e+00, 1.61500e+01, 1.87500e+01,
       8.14800e+01, 1.32350e+00, 1.35290e+00, 1.38230e+00, 1.44117e+00,
       6.50000e-01, 6.60000e-01, 2.66000e+01, 2.64000e+01, 1.52000e+01,
       1.47000e+01, 1.60000e+01, 1.65000e+01, 1.27500e+01, 1.86100e+01,
       2.45200e+01, 2.94200e+01, 7.50000e+01, 1.20000e-01, 7.45950e+01,
       7.91670e+01, 8.32590e+01, 8.59000e+01, 8.70000e+01, 8.75000e+01,
       8.73490e+01, 8.78300e+01, 6.90000e+00, 6.00000e+01, 3.84570e+02,
       3.26600e+02, 3.50000e+01, 4.10000e+01, 5.30000e+01, 3.77000e+01,
       3.70000e+01, 3.29000e+01, 3.36000e+01, 5.48000e+01, 1.80000e-01,
       7.02640e+01, 3.13300e+01, 4.00300e+01, 3.30000e+01, 4.48900e+01,
       5.20000e+01, 5.13850e+01, 5.21800e+01, 5.03000e+01, 5.02750e+01,
       4.89100e+01, 5.10000e+01, 4.88500e+01, 5.06000e+01, 5.25000e+01,
       4.61100e+01, 4.57800e+01, 5.70000e+01, 5.50000e+01, 4.50000e+01,
       1.25900e+00, 8.45500e-01, 6.46000e-01, 1.24000e+00, 8.400

In [47]:
id = df_copy['id']

In [48]:
# Заполним значения с помощью KNN
# Удалим ненужные столбцы из датасета для применения kNN
df_for_kNN = df_copy.drop(columns=['id'])  # Уберите любые другие столбцы, которые вам не нужны для анализа

# Нормализация данных
scaler = StandardScaler()
normalized_data = scaler.fit_transform(df_for_kNN)

# Заполним пропущенные значения в датасете с помощью kNN
imputer = KNNImputer(n_neighbors=5)
imputed_data = imputer.fit_transform(normalized_data)
df_imputed = pd.DataFrame(imputed_data, columns=df_for_kNN.columns)

# Объединим заполненные данные с исходным датасетом
df_filled = pd.concat([df_copy[['id']], df_imputed], axis=1)

# Выведем информацию о пропущенных значениях после заполнения
print(f'Процент пропущенных значений после заполнения: \n{df_filled.isna().sum() * 100 / len(df_filled)}')

Процент пропущенных значений после заполнения: 
id                  0.0
coer_oe             0.0
exc_bias_oe         0.0
fc_field_t          0.0
h_range_max_koe     0.0
length_d_nm         0.0
mr (emu/g)          0.0
sat_em_g            0.0
shape               0.0
space_group_core    0.0
temperature_k       0.0
ver_shift_emu_g     0.0
width_nm            0.0
dtype: float64


In [49]:
print(df_filled['sat_em_g'])

0      0.372228
1      1.109612
2     -0.814302
3     -0.810971
4     -0.484488
         ...   
528   -0.830127
529   -0.829532
530   -0.829365
531   -0.828901
532   -0.838574
Name: sat_em_g, Length: 533, dtype: float64


In [58]:
df_filled

,id,coer_oe,exc_bias_oe,fc_field_t,h_range_max_koe,length_d_nm,mr (emu/g),sat_em_g,shape,space_group_core,temperature_k,ver_shift_emu_g,width_nm
0,1,6.366860,-0.267574,0.589575,0.421082,-0.348494,-0.037091,0.372228,-0.417455,-0.078575,-0.663409,-0.127716,-0.344538
1,2,1.316404,0.114389,0.589575,0.421082,-0.350974,0.359060,1.109612,-0.417455,0.564722,-0.663409,-0.127716,-0.347955
2,3,-0.647012,0.355004,-0.672235,-1.090300,-0.288975,-0.221880,-0.814302,-0.417455,0.597712,1.828333,-0.127716,-0.262533
3,4,-0.630497,0.384909,-0.672235,-1.090300,-0.326174,-0.221692,-0.810971,-0.417455,0.597712,1.828333,-0.127716,-0.313786
4,5,-0.643156,0.332686,-0.672235,-1.090300,-0.319974,-0.217287,-0.484488,-0.417455,0.597712,1.828333,-0.127716,-0.305244
...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,529,-0.364156,-0.115338,-0.167511,0.798928,-0.146379,-0.219896,-0.830127,-0.417455,-2.123929,-0.663409,-0.117288,-0.066063
529,530,-0.335096,-0.213909,0.084851,0.798928,-0.146379,-0.219903,-0.829532,-0.417455,-2.123929,-0.663409,-0.116222,-0.066063
530,531,-0.319313,-0.274150,0.337213,0.798928,-0.177378,-0.219755,-0.829365,-0.417455,-2.123929,-0.663409,-0.113627,-0.108774
531,532,-0.286460,-0.436247,0.589575,0.798928,-0.177378,-0.219762,-0.828901,-0.417455,-2.123929,-0.663409,-0.113812,-0.108774


In [51]:
df_filled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                533 non-null    int64  
 1   coer_oe           533 non-null    float64
 2   exc_bias_oe       533 non-null    float64
 3   fc_field_t        533 non-null    float64
 4   h_range_max_koe   533 non-null    float64
 5   length_d_nm       533 non-null    float64
 6   mr (emu/g)        533 non-null    float64
 7   sat_em_g          533 non-null    float64
 8   shape             533 non-null    float64
 9   space_group_core  533 non-null    float64
 10  temperature_k     533 non-null    float64
 11  ver_shift_emu_g   533 non-null    float64
 12  width_nm          533 non-null    float64
dtypes: float64(12), int64(1)
memory usage: 54.3 KB


In [52]:
# Преобразование обратно в наши значения
org_copy = scaler.inverse_transform(df_filled.iloc[:, 1:]) # Без столбца id
df_recopy = pd.DataFrame(org_copy, columns=df_filled.columns[1:]) # Данные в исходном виде

df_recopy['id'] = id
print(df_recopy['sat_em_g'].unique())

[5.2152400e+01 8.3140000e+01 2.2900000e+00 2.4300000e+00 1.6150000e+01
 1.8750000e+01 8.1480000e+01 1.3235000e+00 1.3529000e+00 1.3823000e+00
 1.4411700e+00 6.5000000e-01 6.6000000e-01 2.6600000e+01 2.6400000e+01
 1.5200000e+01 1.4700000e+01 1.6000000e+01 1.6500000e+01 1.2750000e+01
 1.8610000e+01 2.4520000e+01 2.9420000e+01 7.5000000e+01 1.2000000e-01
 7.4595000e+01 7.9167000e+01 8.3259000e+01 8.5900000e+01 8.7000000e+01
 8.7500000e+01 8.7349000e+01 8.7830000e+01 1.8372000e+01 3.1360000e+01
 5.5691200e+01 2.0040000e+01 3.1040000e+01 6.9000000e+00 6.0000000e+01
 3.8457000e+02 3.2660000e+02 3.5000000e+01 4.1000000e+01 5.3000000e+01
 3.7700000e+01 3.7000000e+01 3.2900000e+01 3.3600000e+01 5.4800000e+01
 1.8000000e-01 7.0264000e+01 3.1330000e+01 4.0030000e+01 3.3000000e+01
 4.4890000e+01 1.3127000e+00 4.3278000e+01 3.0996300e+01 5.2000000e+01
 5.1385000e+01 5.2180000e+01 6.0513000e+01 5.0300000e+01 5.0275000e+01
 4.8910000e+01 5.1000000e+01 4.8850000e+01 5.0600000e+01 5.2500000e+01
 4.611

In [60]:
df_recopy

,coer_oe,exc_bias_oe,fc_field_t,h_range_max_koe,length_d_nm,mr (emu/g),sat_em_g,shape,space_group_core,temperature_k,ver_shift_emu_g,width_nm,id
0,27800.00,-1132.000,5.0,50.0,7.4,13.76705,52.1524,1.0,186.0,5.0,0.0000,7.4,1
1,7800.00,-406.000,5.0,50.0,7.0,43.26420,83.1400,1.0,225.0,5.0,0.0000,7.0,2
2,24.80,51.340,0.0,10.0,17.0,0.00780,2.2900,1.0,227.0,300.0,0.0000,17.0,3
3,90.20,108.180,0.0,10.0,11.0,0.02180,2.4300,1.0,227.0,300.0,0.0000,11.0,4
4,40.07,8.920,0.0,10.0,12.0,0.34980,16.1500,1.0,227.0,300.0,0.0000,12.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,1144.92,-842.644,2.0,60.0,40.0,0.15550,1.6250,1.0,62.0,5.0,0.1125,40.0,529
529,1260.00,-1030.000,3.0,60.0,40.0,0.15500,1.6500,1.0,62.0,5.0,0.1240,40.0,530
530,1322.50,-1144.500,4.0,60.0,35.0,0.16600,1.6570,1.0,62.0,5.0,0.1520,35.0,531
531,1452.60,-1452.600,5.0,60.0,35.0,0.16550,1.6765,1.0,62.0,5.0,0.1500,35.0,532


In [53]:
print(df_recopy.info())
print(df_recopy['space_group_core'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   coer_oe           533 non-null    float64
 1   exc_bias_oe       533 non-null    float64
 2   fc_field_t        533 non-null    float64
 3   h_range_max_koe   533 non-null    float64
 4   length_d_nm       533 non-null    float64
 5   mr (emu/g)        533 non-null    float64
 6   sat_em_g          533 non-null    float64
 7   shape             533 non-null    float64
 8   space_group_core  533 non-null    float64
 9   temperature_k     533 non-null    float64
 10  ver_shift_emu_g   533 non-null    float64
 11  width_nm          533 non-null    float64
 12  id                533 non-null    int64  
dtypes: float64(12), int64(1)
memory usage: 54.3 KB
None
[186. 225. 227.  62. 161. 194. 221.  87. 229.  55. 136.   1. 141. 164.
 160. 212. 123. 217.  63. 213.  15. 182.  58.]


In [54]:
# Теперь мне надо склеить данные до и после заполнеия пропусков
merg_df = pd.merge(df, df_recopy, on='id', suffixes=('_original', '_copy')) # Теперь в датасете есть повторяющиеся колонки, удалим оригинальные с пропусками и сотавим только чистые

print(merg_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 533 entries, 0 to 532
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   c1                         533 non-null    object 
 1   c2                         322 non-null    object 
 2   c3                         9 non-null      object 
 3   chemical_formula           533 non-null    object 
 4   coer_oe_original           510 non-null    float64
 5   crystal_structure          533 non-null    object 
 6   depth_nm                   509 non-null    float64
 7   exc_bias_oe_original       500 non-null    float64
 8   fc_field_t_original        508 non-null    float64
 9   h_range_max_koe_original   526 non-null    float64
 10  id                         533 non-null    int64  
 11  length_d_nm_original       521 non-null    float64
 12  mr (emu/g)_original        430 non-null    float64
 13  sat_em_g_original          420 non-null    float64

In [55]:
# Список колнок, что мы удалим
del_col_list = [col for col in merg_df.columns if col.endswith('_original')]
# Удалим такие колонки 
merg_df = merg_df.drop(columns=del_col_list)

print(merg_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 533 entries, 0 to 532
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   c1                     533 non-null    object 
 1   c2                     322 non-null    object 
 2   c3                     9 non-null      object 
 3   chemical_formula       533 non-null    object 
 4   crystal_structure      533 non-null    object 
 5   depth_nm               509 non-null    float64
 6   id                     533 non-null    int64  
 7   space_group_shell      533 non-null    object 
 8   structure              533 non-null    object 
 9   structure_             533 non-null    object 
 10  formula_combined       533 non-null    object 
 11  coer_oe_copy           533 non-null    float64
 12  exc_bias_oe_copy       533 non-null    float64
 13  fc_field_t_copy        533 non-null    float64
 14  h_range_max_koe_copy   533 non-null    float64
 15  length

In [56]:
# Удалим _copy из названий 
merg_df.columns = merg_df.columns.str.replace('_copy', '')

df = merg_df  
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 533 entries, 0 to 532
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   c1                 533 non-null    object 
 1   c2                 322 non-null    object 
 2   c3                 9 non-null      object 
 3   chemical_formula   533 non-null    object 
 4   crystal_structure  533 non-null    object 
 5   depth_nm           509 non-null    float64
 6   id                 533 non-null    int64  
 7   space_group_shell  533 non-null    object 
 8   structure          533 non-null    object 
 9   structure_         533 non-null    object 
 10  formula_combined   533 non-null    object 
 11  coer_oe            533 non-null    float64
 12  exc_bias_oe        533 non-null    float64
 13  fc_field_t         533 non-null    float64
 14  h_range_max_koe    533 non-null    float64
 15  length_d_nm        533 non-null    float64
 16  mr (emu/g)         533 non

In [38]:
# Определение функции для преобразования строки в объект Composition
def apply_composition(value):
    return Composition(value)

# Применение функции ко всем значениям столбца
df['formula_combined'] = df['c1'].apply(apply_composition)
print(df['formula_combined'].unique())

[Composition('Au1 Fe2 O3') Composition('Fe297 Co3 O400')
 Composition('Fe59 Co1 O80') Composition('Fe29 Co1 O40')
 Composition('Mn15 Fe261 Co24 O400') Composition('Fe27 Co3 O40')
 Composition('Fe5 Co1 O8') Composition('Fe23 Co7 O40')
 Composition('Co1 Fe2 O4') Composition('Cu1 Fe2 O4')
 Composition('Cu2 Fe2 O4') Composition('Mn1 Fe5 O8') Composition('Fe3 O4')
 Composition('Fe12 C5') Composition('Fe2 O3')
 Composition('Fe9 O3 C18 N18') Composition('Fe10 O3 C24 N24')
 Composition('Fe3 O6 Ce1') Composition('Fe3 O6 Mn1')
 Composition('Fe3 O4 Si1') Composition('Fe1 V2 O4')
 Composition('Mg1 Mn9 Fe20 O40') Composition('Mg3 Mn7 Fe20 O40')
 Composition('Mg2 Mn3 Fe10 O20') Composition('Mg1 Mn1 Fe4 O8')
 Composition('Mg1 Fe2 O4') Composition('Mn1 Zn1 Fe4 O8')
 Composition('Mn1 Fe2 O4') Composition('Zn1 Fe4 Ni1 O8')
 Composition('Zn7 Fe41 Ni13 O80') Composition('Zn7 Fe41 Ni13 O100')
 Composition('Zn7 Fe41 Ni13 O120') Composition('Zn7 Fe41 Ni13 O140')
 Composition('Ni1 Fe2 O4') Composition('Co50 N

In [39]:
# Выгрузим данные отдельный датасет 
df.to_csv(r'C:\Users\Ксения\Project\Magnet_Exchange_bias\data\part2_3.csv')